In [1]:
import pandas as pd
from ast import literal_eval

In [2]:
df = pd.read_pickle("../results/schembl_summs_v3_alg_cleaned_labels.pkl")

In [3]:
vocab_df = pd.read_csv("../results/schembl_summs_v4_vocab_gpt_cleaned_eps_0.340_diff_20030_clusters_errors_fixed.csv")
# remove row with API REQUEST ERROR
vocab_df = vocab_df[vocab_df["gpt_cleaned_labels"] != "API REQUEST ERROR"]

In [4]:
# make each element in original_clustered_labels its own row, repeating the gpt_cleaned_labels
vocab_df = vocab_df.assign(original_clustered_labels=vocab_df['original_clustered_labels'].str.split(',')).explode('original_clustered_labels')
vocab_df = vocab_df.reset_index()
# convert to dictionary
vocab_dict = dict(zip(vocab_df["original_clustered_labels"].tolist(), vocab_df["gpt_cleaned_labels"].tolist()))

In [6]:
df["summarizations"] = df["summarizations"].map(set)

In [10]:
# map vocab_dict to each value in summarizations, ignoring when the value is not in vocab_dict and defaulting to the original value
df['summarizations'] = df['summarizations'].map(lambda x: [vocab_dict.get(i, i) for i in x])

In [12]:
df.columns

Index(['smiles', 'cid', 'patent_ids', 'summarization_sources',
       'summarizations'],
      dtype='object')

In [15]:
df[["smiles", "cid", "patent_ids", "summarizations"]].to_csv("../results/schembl_summs_v4_gpt_cleaned_eps_0.340_diff_20030_final.csv")